# 1. Cython wrapper nad C funkcí
Cython umožní volat C funkci s minimálním overheadem. Důležité je držet stejné ABI (Application Binary Interface) jako v C: `float32` buffery a rozměry typu `size_t`.


## 1.1 Co znamenají značky v Cython buňce
V ukázce se objeví několik „tagů“, které je potřeba znát:

- `%load_ext Cython`: načte IPython rozšíření pro Cython magic buňky,
- `%%cython -I .`: přeloží aktuální buňku jako Cython; `-I .` přidá aktuální složku do include cesty,
- `# distutils: sources = matrix_vector_multiply.c`: přidá C zdrojový soubor do buildu,
- `# cython: language_level=3`: použije pravidla Pythonu 3 při překladu Cythonu,
- `cnp.ndarray[..., mode="c"]`: vyžaduje C-contiguous NumPy pole,
- `cdef extern from ...`: deklarace C funkce dostupné pro volání z Cythonu.


In [ ]:
import os
import sys
import numpy as np

os.environ["PATH"] = f"{os.path.dirname(sys.executable)}:{os.environ['PATH']}"

matrix = np.ascontiguousarray([[1, 2, 3], [4, 5, 6]], dtype=np.float32)
vector = np.ascontiguousarray([1, 2, 3], dtype=np.float32)
result = np.zeros(2, dtype=np.float32)


In [ ]:
%load_ext Cython

In [ ]:
%%cython -I .
# distutils: sources = matrix_vector_multiply.c
# cython: language_level=3

cimport numpy as cnp
from libc.stddef cimport size_t

cdef extern from "matrix_vector_multiply.h":
    void matrix_vector_multiply(
        const float *matrix,
        const float *vector,
        float *result,
        size_t rows,
        size_t cols,
    )


def mvp_func_cy(
    cnp.ndarray[cnp.float32_t, ndim=2, mode="c"] matrix,
    cnp.ndarray[cnp.float32_t, ndim=1, mode="c"] vector,
    cnp.ndarray[cnp.float32_t, ndim=1, mode="c"] result,
):
    cdef size_t rows = matrix.shape[0]
    cdef size_t cols = matrix.shape[1]

    if vector.shape[0] != cols:
        raise ValueError("Počet sloupců matice musí odpovídat délce vektoru.")
    if result.shape[0] != rows:
        raise ValueError("Výstupní vektor má špatnou délku.")

    matrix_vector_multiply(&matrix[0, 0], &vector[0], &result[0], rows, cols)


In [ ]:
mvp_func_cy(matrix, vector, result)
print("Cython magic:", result)
print("NumPy:", matrix @ vector)


## 1.2 Stejný wrapper přes `setup.py`
V této variantě překládáme Cython modul klasicky přes `setup.py`.

In [ ]:
%%writefile mvp_cy.pyx
cimport numpy as cnp
from libc.stddef cimport size_t

cdef extern from "matrix_vector_multiply.h":
    void matrix_vector_multiply(
        const float *matrix,
        const float *vector,
        float *result,
        size_t rows,
        size_t cols,
    )


def mvp_func_cy(
    cnp.ndarray[cnp.float32_t, ndim=2, mode="c"] matrix,
    cnp.ndarray[cnp.float32_t, ndim=1, mode="c"] vector,
    cnp.ndarray[cnp.float32_t, ndim=1, mode="c"] result,
):
    cdef size_t rows = matrix.shape[0]
    cdef size_t cols = matrix.shape[1]

    if vector.shape[0] != cols:
        raise ValueError("Počet sloupců matice musí odpovídat délce vektoru.")
    if result.shape[0] != rows:
        raise ValueError("Výstupní vektor má špatnou délku.")

    matrix_vector_multiply(&matrix[0, 0], &vector[0], &result[0], rows, cols)


In [ ]:
%%writefile setup.py
from setuptools import Extension, setup
from Cython.Build import cythonize
import numpy as np

ext_modules = [
    Extension(
        "mvp_cy",
        ["mvp_cy.pyx", "matrix_vector_multiply.c"],
        include_dirs=[np.get_include(), "."],
        extra_compile_args=["-O3", "-std=c11"],
        language="c",
    )
]

setup(
    name="mvp_cy",
    ext_modules=cythonize(ext_modules, language_level="3"),
)


In [ ]:
!python setup.py build_ext --inplace

In [ ]:
import importlib
import mvp_cy

importlib.reload(mvp_cy)


In [ ]:
result2 = np.zeros(2, dtype=np.float32)
mvp_cy.mvp_func_cy(matrix, vector, result2)
print("Cython modul:", result2)
print("NumPy:", matrix @ vector)
